In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time
import re
import numpy as np
from tqdm import tqdm

movies = pd.read_excel('movie15.xls', skiprows=4)
movies.head()

,영화명,영화명(영문),제작연도,제작국가,유형,장르,제작상태,감독,제작사
0,부활의 노래,The Song Of Resurrection,1990.0,한국,장편,드라마,개봉,이정국,새빛영화제작소
1,젖은장모-사위 대물에 눈 뒤집히네,NaN,2022.0,일본,장편,"성인물(에로),드라마,멜로/로맨스",개봉,츠지 코지,NaN
2,점장님 팬티 내릴게요 눈감아 주세요,NaN,2022.0,일본,장편,"성인물(에로),드라마,멜로/로맨스",개봉,타카야마 켄타로,NaN
3,옆집여자-남편 대신 시원하게 싸줬다,NaN,2023.0,일본,장편,"성인물(에로),드라마,멜로/로맨스",개봉,타카야마 켄타로,NaN
4,물 넘치는 섹파-방금 쌌는데 또 싸고싶어,NaN,2023.0,일본,장편,"성인물(에로),드라마,멜로/로맨스",개봉,히무 롯쿠,NaN


In [2]:
def genre(x):
    if '성인물(에로)' not in x:
        return x
movies['장르'] = movies.장르.apply(genre)
movies = movies[~movies['장르'].isnull()].reset_index(drop=True)

In [3]:
print(len(movies.장르.unique()))
movies.head()

885


,영화명,영화명(영문),제작연도,제작국가,유형,장르,제작상태,감독,제작사
0,부활의 노래,The Song Of Resurrection,1990.0,한국,장편,드라마,개봉,이정국,새빛영화제작소
1,이매큘레이트,Immaculate,2024.0,미국,장편,공포(호러),개봉,마이클 모한,NaN
2,러브 라이즈 블리딩,Love Lies Bleeding,2023.0,미국,장편,"범죄,멜로/로맨스",개봉,로즈 글래스,NaN
3,짱,Top Dog,2018.0,한국,장편,액션,개봉,송재덕,"(주)영화사 사계절,펀펀한영화사"
4,싸움의 기술 2,The Art of Fighting 2,2020.0,한국,장편,액션,개봉,김희성,(주)영화사 사계절


In [4]:
movies = movies.영화명.tolist()
movies[:5]

['부활의 노래', '이매큘레이트', '러브 라이즈 블리딩', '짱', '싸움의 기술 2']

# 범죄도시

스크래핑이 안되는 건 아닌데 영화 평점이 없는 데이터는 추가가 안돼서 문제임;;

In [5]:
url = 'https://namu.wiki/w/' # %EC%82%B4%EC%9D%B8%EC%9D%98%20%EC%B6%94%EC%96%B5
pd_movies = []
content = ''
cnt = 0
for movie in tqdm(movies):
    try:
        page = requests.get(url + movie)
        html = BeautifulSoup(page.text, 'html.parser')

        soup1 = html.select('#app div div div div') # 시놉시스
        soup2 = html.select('#app div div div div .wiki-paragraph') # 테이블 형식 데이터 수집

        evaluation = []
        st = 0

        for s in soup1:
            if s.text != '':
                if '시놉시스[편집]' in s.text:
                    content = s.text
                    break
        lst = content.split('[편집]')
        for i, v in enumerate(lst):
            if v[-4:] == '시놉시스':
                synopsys = lst[i+1]
                break
        for i, v in enumerate(soup1):
            if '평가[편집]' in v.text:
                evaluation.append(soup1[i:i+10])
        evaluation = evaluation[-1]
    except:
        page = requests.get(url + movie +'(영화)')
        html = BeautifulSoup(page.text, 'html.parser')

        soup1 = html.select('#app div div div div') # 시놉시스
        soup2 = html.select('#app div div div div .wiki-paragraph') # 테이블 형식 데이터 수집

        evaluation = []
        st = 0

        for s in soup1:
            if s.text != '':
                if '시놉시스[편집]' in s.text:
                    content = s.text
                    break
        lst = content.split('[편집]')
        for i, v in enumerate(lst):
            if v[-4:] == '시놉시스':
                synopsys = lst[i+1]
                break
        for i, v in enumerate(soup1):
            if '평가[편집]' in v.text:
                evaluation.append(soup1[i:i+10])
    
    try:
        evaluation = evaluation[-1]
        lst = [] # 영화에 대한 정보를 담는 리스트
        for s in soup2[1:]: # 영화의 정보, 1은 총 데이터 묶음이므로 생략
            if s.text != '':
                if s.text == '장르':
                    st = 1
                elif '개요' in s.text:
                    st = 0
                    break
                if st == 1:
                    lst.append(s.text)
            if '개요' in s.text:
                break

        table_data = []
        data = []
        idx = -1

        if len(evaluation[idx]) == 0: # 페이지마다 스크래핑되는 경우와 안되는 경우가 있음.
            idx = -2                  # 인덱스를 바꾸면 가져올 수 있음

        for i in evaluation[idx].select('a'):
            if ('http' not in i.attrs['title']) & (i.attrs['title'] != '') & (len(i.attrs['title']) < 30):
                try:
                    if '네이버 영화' in table_data[-1]: # 네이버 영화는 동일한 데이터가 2번 입력됨
                        del data[0]
                except:
                    pass
                table_data.append(data)
                table_data.append(i.attrs['title'])
                data = []
            if (i.text != '') & bool(re.search(r'[\d%/]', i.text)) & (len(data) < 2) & ~bool(re.search(r'[\[\]]', i.text)): 
                # 데이터값에는 다양한 데이터가 들어가지만 주로 1/10 같은 분수 표현과 숫자로 구성 되어 있음. 
                # 또한 %도 들어가며 데이터는 최대 2개만 들어가는 것으로 보임
                data.append(i.text)

        
        try: # max_length 변수 참고
            # 평가 기관 및 점수 데이터 가공
            table_data = table_data[1:] # 첫째 데이터는 비어있음
            eval_dict = dict(zip(table_data[::2],table_data[1::2])) # dict화
            
            max_length = max(len(v) for v in eval_dict.values()) # 평가 챕터가 열려있지 않으면 에러 발생하므로 try문으로 묶음
            for k, v in eval_dict.items(): # 데이터 길이 맞추기
                if len(v) < max_length:
                    eval_dict[k] = v + [np.nan] * (max_length - len(v))

            eval_df = pd.DataFrame(eval_dict) # 데이터프레임화
            eval_df = eval_df.dropna(axis=1, how='all') # 데이터가 전부 비어있으면 삭제
            eval_df = eval_df.transpose().reset_index()
            eval_df.columns = ['평가', '점수1', '점수2']

            # 영화 정보 가공
            lst.insert(0, '영화명')
            lst.insert(1, movie)
            df = pd.DataFrame(lst)
            df = pd.concat([df.iloc[::2].reset_index(drop=True), df.iloc[1::2].reset_index(drop=True)], axis=1)
            df.columns = ['kind', 'content']
            df.set_index(df.kind, inplace=True)
            df = df.drop('kind', axis=1)
            pd_movies.append([eval_df,df])
            
        except Exception:
            pass
    except:
        pass
    cnt += 1
    if cnt == 50:
        break

  0%|          | 49/13306 [00:37<2:49:01,  1.31it/s]


49개중 4개만 데이터가 담김;;

In [11]:
len(pd_movies)

4

In [207]:
evaluation

[<div data-v-95bee609="" style="b10dfw+X"><h3 class="wiki-heading" data-v-95bee609=""><a data-v-95bee609="" href="#toc" id="s-8.1">8.1.</a> <span data-v-95bee609="" id="평론가 평가">평론가 평가<span class="wiki-edit-section" data-v-95bee609=""><a data-v-95bee609="" href="/edit/%ED%83%88%EC%A3%BC(%EC%98%81%ED%99%94)?section=10" rel="nofollow">[편집]</a></span></span></h3></div>,
 <div data-v-95bee609="" style="b10dfw+X"></div>,
 <div data-v-95bee609="" style="b10dfw+X"></div>,
 <div class="ou9T9n3p" data-v-95bee609="" data-v-befa586c=""><div data-v-95bee609="" style="ou9T9n3p"></div><div data-v-95bee609="" style="ou9T9n3p"><div data-v-95bee609="" style="ou9T9n3p"></div><div data-v-95bee609="" style="ou9T9n3p"><div class="wiki-heading-content" data-v-95bee609=""><blockquote class="wiki-quote" data-v-95bee609=""><div class="wiki-paragraph" data-v-95bee609=""><strong data-v-95bee609="">실패할 걸 알면서도 우선 달리고 보는 두 남자를 꼭 닮은 영화</strong></div><hr data-v-95bee609=""/><div class="wiki-paragraph" data-v-95bee609=

In [200]:
for i in evaluation[-1].select('a'):
    if ('http' not in i.attrs['title']) & (i.attrs['title'] != '') & (len(i.attrs['title']) < 30):
        try:
            if '네이버 영화' in table_data[-1]: # 네이버 영화는 동일한 데이터가 2번 입력됨
                del data[0]
        except:
            pass
        print(i.attrs)
        table_data.append(data)
        table_data.append(i.attrs['title'])
        data = []
    if (i.text != '') & bool(re.search(r'[\d%/]', i.text)) & (len(data) < 2) & ~bool(re.search(r'[\[\]]', i.text)): 
        # 데이터값에는 다양한 데이터가 들어가지만 주로 1/10 같은 분수 표현과 숫자로 구성 되어 있음. 
        # 또한 %도 들어가며 데이터는 최대 2개만 들어가는 것으로 보임
        data.append(i.text)
data

[]

In [192]:
lst

['장르',
 '액션, 스릴러, 밀리터리, 드라마',
 '감독',
 '이종필',
 '각본',
 '권성휘[1], 김우근',
 '각색',
 '이종필, 김민석, 문병곤',
 '제작',
 '박은경, 권성휘, 손다올',
 '제작투자',
 '홍정인',
 '프로듀서',
 '강대욱, 김우성',
 '촬영',
 '김성안',
 '조명',
 '이승빈',
 '미술',
 '배정윤',
 '분장',
 '김현정',
 '편집',
 '이강희',
 '음악',
 '달파란',
 '시각효과',
 'RASCA',
 '출연',
 '이제훈, 구교환, 홍사빈 外',
 '제작사',
 '더램프, 은하수필름[2]',
 '배급사',
 ' 플러스엠',
 '개봉일',
 ' 2024년 7월 3일',
 '촬영 기간',
 '2022년 2월 20일 ~ 2022년 6월 7일',
 '화면비',
 '2.39:1',
 '상영 타입',
 '2D, Dolby Atmos',
 '상영 시간',
 '94분 (1시간 34분 9초)',
 '제작비',
 '85억 원[3]',
 '대한민국 총 관객 수',
 '1,616,568명 (2024년 7월 19일 기준)',
 '상영 등급',
 ' 12세 이상 관람가[4]']

In [190]:
pd_movies

[[                  평가                                                점수1  \
  0              메타크리틱                             메타스코어 <메타스코어 점수> / 100   
  1             로튼 토마토                                  신선도 <토마토미터 퍼센티지>%   
  2               IMDb                                    평점 <유저 평점> / 10   
  3         Letterboxd                                      별점 <평점> / 5.0   
  4               알로시네                              전문가 별점 <전문가 평점> / 5.0   
  5             키노포이스크                                       별점 <평점> / 10   
  6             야후! 재팬                                      별점 <평점> / 5.0   
  7           Filmarks                                      별점 <평점> / 5.0   
  8                엠타임                                       평점 <평점> / 10   
  9                더우반                                       별점 <평점> / 10   
  10             왓챠피디아                                      별점 <평점> / 5.0   
  11             키노라이츠                                           지수 91.3%   

In [42]:
# 기존 코드
# for i in evaluation[-1].select('a'):
#     if ('http' not in i.attrs['title']) & (i.attrs['title'] != ''):
#         print('title',i.attrs['title'])
#     if (i.text != ''):
#         print(i.text)

# 기존 작업

In [ ]:
pd_movies = []
for movie in movies:
    try:
        url = 'https://namu.wiki/w/' # %EC%82%B4%EC%9D%B8%EC%9D%98%20%EC%B6%94%EC%96%B5

        page = requests.get(url + movie)
        html = BeautifulSoup(page.text, 'html.parser')
        # html.select('#app > div > div.MMT57cZa.iHNYoQlC > div > div')
        soup = html.select('#app div div div div .wiki-paragraph') # .wiki-paragraph

        time.sleep(1) # 1초동안 정지
        
        lst = [] # 영화에 대한 정보를 담는 리스트
        st = 0
        for s in soup:
            if s.text != '':
                if s.text == '장르':
                    st = 1
                elif '개요' in s.text:
                    st = 0
                    break
                if st == 1:
                    lst.append(s.text)
            if '개요' in s.text:
                break
        lst.insert(0, '영화명')
        lst.insert(1, movie)
        df = pd.DataFrame(lst)
        df = pd.concat([df.iloc[::2].reset_index(drop=True), df.iloc[1::2].reset_index(drop=True)], axis=1)
        df.columns = ['kind', 'content']
        df.set_index(df.kind, inplace=True)
        df = df.drop('kind', axis=1)
        pd_movies.append(df)
    except Exception as e:
        print(e)

In [58]:
pd_movies[5]

,content
kind,
영화명,우마무스메 프리티 더비 새로운 시대의 문
장르,"경마, 스포츠"
감독,야마모토 켄
시나리오 디렉터,코바리 테츠야(小針哲也)
스토리 원안,콘티
야마모토 켄이시이 토시마사[1]나카야마 나오야[L],연출
야마모토 켄후치모토 슈헤이쿠로야나기 토시마사랴오 청즈나카야마 나오야[L]오카모토 마나부연출 보좌,각본
요시무라 키요코(吉村清子),원작
Cygames,제작


In [ ]:
# https://jalammar.github.io/illustrated-word2vec/
# 해당 주소를 참고하여 유사한 단어를 알아야함
# https://littlefoxdiary.tistory.com/42 자연어 처리 데이터셋 목록
# https://github.com/Espresso-AI/bertsum-korean?tab=readme-ov-file 텍스트 요약 모델 이용

In [ ]:
# 문장이 들어왔을 때
# 불용어 처리
# 명사(영화명 포함)
# 명사와 유사한 단어의 장르를 탐색(혹은 영화)
# 질문과 유사한 줄거리인지 어떻게 판단하나?
# ex)로봇이 나와서 이기는 영화?
# ex)열린 결말의 영화를 추천해줘